# 代码部分

In [1]:
# 数据库连接
import psycopg2
import copy
conn = psycopg2.connect(database="maine", user="postgres", password="1997pgsql", host="127.0.0.1", port="5432")
print("Connect database successfully") 
cur=conn.cursor()

Connect database successfully


In [2]:
conn.rollback()

# 根据关键词选出作为路径的way，新建一张表保存

In [5]:
%%time
cur.execute('drop table if exists qualified_way_nodes')
sql="""
create table qualified_way_nodes as select *
    from way_nodes
 	where way_id in (
		select distinct way_id
 		from way_tags
 		where k='highway' )
"""
cur.execute(sql)
conn.commit()
cur.execute("select count(distinct way_id) as count from qualified_way_nodes")
res=cur.fetchall()[0][0]
print('extract qualified ways done. there are total %d ways'%(res))
# k='cycleway' or k='oneway' or k='highway'or k='bridge' or k='bike' or k='busway' or k='expressway'or k like '%bus%' or k like 'car%' or k='horse' or k='railway' or k='footway' or k='sidewalk' or k='driveway' or k='way' or k='runway' or k='bicycle'

extract qualified ways done. there are total 184242 ways
Wall time: 7.64 s


# 计算路径是否可以组成一个连通图

In [6]:
import random

node_id_list=[]
cur.execute("select node_id from qualified_way_nodes limit 1000")
res=cur.fetchall()
for item in res:
    node_id_list.append(item[0])
seed_node_id=random.choice(node_id_list)

way_id_set=set()
node_id_set=set()
node_id_set.add(seed_node_id)

In [7]:
%%time
while 1:
    print('number of new node_id to be query:',len(node_id_set))
    sql_str="("
    for node in node_id_set:
        sql_str+=str(node)
        sql_str+=','
    sql_str=sql_str[:-1]
    sql_str+=")"
    node_id_set=set()
    if sql_str==")":
        break
    cur.execute("select distinct way_id from qualified_way_nodes where node_id in"+sql_str)
    res=cur.fetchall()
    new_way_id_set=set()
    for item in res:
        new_way_id_set.add(item[0])
    if new_way_id_set-way_id_set==False:
        break
    candidate_way_id_set=new_way_id_set-way_id_set
    way_id_set=way_id_set|new_way_id_set
    print('the size of way_id_set is:%d'%(len(way_id_set)))
    
    new_node_id_set=set()
    print('number of new way_id to be query:',len(candidate_way_id_set))
    if len(candidate_way_id_set)==0:
        break
    sql_str="("
    for way_id in candidate_way_id_set:
        sql_str+=str(way_id)
        sql_str+=','
    sql_str=sql_str[:-1]
    sql_str+=")"
    cur.execute("select distinct node_id from qualified_way_nodes where way_id in"+sql_str)
    result=cur.fetchall()
    for item in result:
        new_node_id_set.add(item[0])
    node_id_set=new_node_id_set
print(len(way_id_set))

number of new node_id to be query: 1
the size of way_id_set is:1
number of new way_id to be query: 1
number of new node_id to be query: 9
the size of way_id_set is:2
number of new way_id to be query: 1
number of new node_id to be query: 19
the size of way_id_set is:4
number of new way_id to be query: 2
number of new node_id to be query: 94
the size of way_id_set is:20
number of new way_id to be query: 16
number of new node_id to be query: 163
the size of way_id_set is:35
number of new way_id to be query: 15
number of new node_id to be query: 443
the size of way_id_set is:81
number of new way_id to be query: 46
number of new node_id to be query: 927
the size of way_id_set is:210
number of new way_id to be query: 129
number of new node_id to be query: 2016
the size of way_id_set is:372
number of new way_id to be query: 162
number of new node_id to be query: 2573
the size of way_id_set is:558
number of new way_id to be query: 186
number of new node_id to be query: 3687
the size of way_id_

number of new node_id to be query: 32704
the size of way_id_set is:156190
number of new way_id to be query: 1666
number of new node_id to be query: 30272
the size of way_id_set is:157915
number of new way_id to be query: 1725
number of new node_id to be query: 29744
the size of way_id_set is:159534
number of new way_id to be query: 1619
number of new node_id to be query: 30405
the size of way_id_set is:161119
number of new way_id to be query: 1585
number of new node_id to be query: 31992
the size of way_id_set is:162692
number of new way_id to be query: 1573
number of new node_id to be query: 31930
the size of way_id_set is:164307
number of new way_id to be query: 1615
number of new node_id to be query: 28912
the size of way_id_set is:165926
number of new way_id to be query: 1619
number of new node_id to be query: 29113
the size of way_id_set is:167680
number of new way_id to be query: 1754
number of new node_id to be query: 29971
the size of way_id_set is:169268
number of new way_id t

In [8]:
all_way_id=set()
cur.execute("select way_id from qualified_way_nodes")
res=cur.fetchall()
for item in res:
    all_way_id.add(item[0])
disqualified_way_id=all_way_id-way_id_set
len(disqualified_way_id)

6248

In [9]:
# 删除那些不连通的way_id
sql_delete="delete from qualified_way_nodes where way_id in ("
for wayid in disqualified_way_id:
    sql_delete+=str(wayid)
    sql_delete+=","
sql_delete=sql_delete[:-1]
sql_delete+=")"
cur.execute(sql_delete)
conn.commit()
cur.execute("select count(distinct way_id) as count from qualified_way_nodes")
res=cur.fetchall()[0][0]
print('filtered qualified ways done. there are total %d ways'%(res))

filtered qualified ways done. there are total 177994 ways


# 选出作为结点的node，新建一张表来保存

In [3]:
%%time 
cur.execute('drop table if exists qualified_nodes')
sql="""
create table qualified_nodes as select id,geom 
from nodes 
where nodes.id in (
	select node_id
 	from qualified_way_nodes )
"""
cur.execute(sql)
conn.commit()
cur.execute("select count(id) as count from qualified_nodes")
res=cur.fetchall()[0][0]
print('extact qualified nodes node. there are total %d nodes.'%(res))

extact qualified nodes node. there are total 2389361 nodes.
Wall time: 8.93 s


# 选出作为POI的node,新建一张表来保存

In [31]:
%%time
cur.execute('drop table if exists qualified_poi')
cur.execute('create table qualified_poi (node_id bigint not null,k text,category_id int)')
conn.commit()
category_sql_list=["k like 'payment%'","k like 'fuel%'","k like 'lunch%'","k like 'buffet%'","k = 'studio'","k like 'service%'",
                  "k ='vending'","k like 'school%'","k='beer'","k='restaurant' ","k like 'drink%' ","k like 'toilet%'","k like 'marketplace%'",
                   " k like 'cuisine%'","k ='brewery'","k='consulate'","k='sauna'","k='sculptor'","k='distillery'","k='furniture'",
                  "k='pastry'","k like 'theatre%'","k='university'","k='tower'","k='military'","k='store'","k='clothes'","k='lottery'",
                  "k='breakfast'","k='office' ","k='grave'"," k='parking'","k='police'","k='dinner'","k='golf'","k='preschool'","k='church'",
                  "k='coffee'"," k='atm'","k='shop'","k='massage'","k='museum'","k='bakery'","k='park'","k='hospital'","k='jewelry'",
                  "k='goods'","k='farm'","k='artwork' ","k='amenity'","k='healthcare' ","k='karaoke'","k='cafe'"]
select_poi_sql="""
insert into qualified_poi
select node_id,k ,0
from node_tags 
where %s
"""
set_category_id_sql = """
update qualified_poi set category_id=%d where %s
"""
category_count=0
for i in range(len(category_sql_list)):
    try:
        category_string=category_sql_list[i]
        cur.execute(select_poi_sql%(category_sql_list[i]))
        cur.execute(set_category_id_sql%(i,category_sql_list[i]))
        conn.commit()
        print('category %d done.'%(i))
    except Exception as e:
        print(e)
        conn.rollback()
print('extract qualified poi done.')

category 0 done.
category 1 done.
category 2 done.
category 3 done.
category 4 done.
category 5 done.
category 6 done.
category 7 done.
category 8 done.
category 9 done.
category 10 done.
category 11 done.
category 12 done.
category 13 done.
category 14 done.
category 15 done.
category 16 done.
category 17 done.
category 18 done.
category 19 done.
category 20 done.
category 21 done.
category 22 done.
category 23 done.
category 24 done.
category 25 done.
category 26 done.
category 27 done.
category 28 done.
category 29 done.
category 30 done.
category 31 done.
category 32 done.
category 33 done.
category 34 done.
category 35 done.
category 36 done.
category 37 done.
category 38 done.
category 39 done.
category 40 done.
category 41 done.
category 42 done.
category 43 done.
category 44 done.
category 45 done.
category 46 done.
category 47 done.
category 48 done.
category 49 done.
category 50 done.
category 51 done.
category 52 done.
extract qualified poi done.
Wall time: 4.29 s


# 记录每个node在记录中的出现频率，将用来确定是否作为路径交点

In [4]:
%%time
cur.execute('drop table if exists node_freq')
#cur.execute('create table node_freq(node_id bigint not null primary key,freq bigint)')
conn.commit()
sql="create table node_freq as select node_id, count(way_id) as freq from qualified_way_nodes group by node_id"
cur.execute(sql)
conn.commit()
print('count node frequency done.')

count node frequency done.
Wall time: 6.45 s


# 构建路网数据集主要代码

In [5]:
# 方法定义
from math import radians, cos, sin, asin, sqrt
import time
import random
 
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000

def calculate_road_length(node_list):
    node_dist_dict=dict()
    sql_str="("
    for n in node_list:
        sql_str+=str(n)
        sql_str+=','
    sql_str=sql_str[:-1]
    sql_str+=")"
    try:
        cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id in "+sql_str)
        result=cur.fetchall()
        for item in result:
            node_dist_dict[item[0]]=(item[1],item[2])

        length=0.0
        nodeA=[node_list[0],0.0,0.0]
        #cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id=%d"%(int(nodeA[0])))
        #nodeAres=cur.fetchall()[0]
        nodeA[1]=node_dist_dict[nodeA[0]][0]
        nodeA[2]=node_dist_dict[nodeA[0]][1]
        node_list_len=len(node_list)
        for i in range(1,node_list_len):
            nodeB=[node_list[i],0.0,0.0]
            #cur.execute("select id, st_x(geom) as lon,st_y(geom) as lat from qualified_nodes where id=%d"%(int(nodeB[0])))
            #nodeBres=cur.fetchall()[0]
            nodeB[1]=node_dist_dict[nodeB[0]][0]
            nodeB[2]=node_dist_dict[nodeB[0]][1]
            #print(nodeA[0],nodeA[1],nodeA[2],nodeB[0],nodeB[1],nodeB[2])
            distance_res=haversine(nodeA[1],nodeA[2],nodeB[1],nodeB[2])
            #print(distance_res)
            length+=distance_res
            nodeA=copy.deepcopy(nodeB)
        return length
    except Exception as e:
        print('in calculate_road_length(): ',e)
        return 0
    

def renumber_node_id(origin_node_id,next_rank_id): 
    try:
        cur.execute("select node_new_id from node_numbers where node_id=%d"%(origin_node_id))
        result=cur.fetchall()
        if len(result)==0:
            cur.execute("insert into node_numbers values(%d,%d)"%(origin_node_id,next_rank_id))
        return cur.fetchall()[0][0]
    except:
        print('in renumber_node_id() : ',e)
        return origin_node_id
    
def write_latlon(fileObject,nodeid,nodeidNew):
    cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id="+str(nodeid))
    result=cur.fetchall()
    longitude=result[0][1]
    latitude=result[0][2]
    fileObject.write(str(nodeidNew)+" "+str(longitude)+" "+str(latitude)+"\n")

In [6]:
cur.execute("select distinct way_id from qualified_way_nodes")
res_qualified_way_id=cur.fetchall()
print("there are %d records in total."%(len(res_qualified_way_id)))

there are 177994 records in total.


## 没有进行路网压缩,且没有POI

In [7]:
%%time
map_file=open('../../map_maine/DATA_MA_0.txt','w')
latlon_file=open('../../map_maine/LATLON_MA_0.txt','w')
#res_qualified_way_id=[[192046993,]]
count=0
renumbered_node=dict()
for row in res_qualified_way_id:
    if count%100==0:
        print('building the %d th road.'%(count))
    count+=1

    #start=time.perf_counter()
    qualified_way_id=row[0]
    cur.execute("select * from qualified_way_nodes where way_id=%d order by sequence_id"%(qualified_way_id))
    res_way_node=cur.fetchall() # [[way_id,node_id,sequenced_id],[],[]]
    #end=time.perf_counter()
    #print("time for select way nodes:",end-start)

    isFirst=True
    start_node=0
    start_node_renumber=0
    start_n_lonlat=[0,0,0]
    end_node=0
    end_n_lon_lat=[0,0,0]
    end_node_renumber=0
    #print(res_way_node)
    for item in res_way_node:
        if isFirst:
            start_node=item[1]
            cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id ="+str(start_node))
            result=cur.fetchall()
            start_n_lonlat=result[0]
            if start_node in renumbered_node:
                # 已经被重新编码过，在字典中获取之前的编码
                start_node_renumber=renumbered_node[start_node]
            else:
                # 没有被重新编码，就赋予新编码
                next_node_id=len(renumbered_node)
                renumbered_node[start_node]=next_node_id
                start_node_renumber=next_node_id
            latlon_file.write(str(start_node_renumber)+" "+str(start_n_lonlat[1])+" "+str(start_n_lonlat[2])+"\n")
            isFirst=False
            continue
        end_node=item[1]

        #start=time.perf_counter()
        cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id ="+str(end_node))
        result=cur.fetchall()
        #end=time.perf_counter()
        #print("time for select lonlat:",end-start)
        end_n_lonlat=result[0]
        node_list=[start_node,end_node]
        #print(start_n_lonlat,"   ",end_n_lonlat)

        #start=time.perf_counter()
        distance=haversine(start_n_lonlat[1],start_n_lonlat[2],end_n_lonlat[1],end_n_lonlat[2])
        #end=time.perf_counter()
        #print("time for computing distance:",end-start)

        #start=time.perf_counter()
        if end_node in renumbered_node:
            # 已经被重新编码过，在字典中获取之前的编码
            end_node_renumber=renumbered_node[end_node]
        else:
            # 没有被重新编码，就赋予新编码
            next_node_id=len(renumbered_node)
            renumbered_node[end_node]=next_node_id
            end_node_renumber=next_node_id
        #end=time.perf_counter()
        #print("time for renumber:",end-start)

        map_file.write(str(start_node_renumber)+' '+str(end_node_renumber)+' '+str(distance)
                           +' '+'0'+'\n')
        latlon_file.write(str(end_node_renumber)+" "+str(end_n_lonlat[1])+" "+str(end_n_lonlat[2])+"\n")    
        start_node=end_node
        start_n_lonlat=end_n_lonlat
        start_node_renumber=end_node_renumber


map_file.close()
latlon_file.close()
print("the total node number is %d"%(len(renumbered_node)))
        

building the 0 th road.


KeyboardInterrupt: 

## 路网压缩版本

In [ ]:
%%time
map_file=open('../../map_maine/DATA_MA_0.txt','w')
latlon_file=open('../../map_maine/LATLON_MA_0.txt','w')
#res_qualified_way_id=[[192046993,]]
count=0
renumbered_node=dict()
for row in res_qualified_way_id:
    if count%100==0:
        print('building the %d th road.'%(count))
    count+=1
    qualified_way_id=row[0]

    #start=time.perf_counter()
    cur.execute("select * from qualified_way_nodes where way_id=%d order by sequence_id"%(qualified_way_id))
    res_way_node=cur.fetchall() # (way_id,node_id,sequenced_id)
    #end=time.perf_counter()
    #print('select node from qualified_way_nodes time consume: %f seconds.'%(end-start))
    res_len = len(res_way_node)
    road_length=0
    numofPOI=0
    road_list=[]
    curr_road_nodes=[]
    #print(res_way_node)
    start_node=res_way_node[0][1]
    end_node=0
    curr_road_nodes.append(start_node)

    #start=time.perf_counter()
    # 对每个node_id 进行处理,拆成多条road放到road list中
    sql_str="("
    for i in res_way_node:
        sql_str+=str(i[1])
        sql_str+=','
    sql_str=sql_str[:-1]
    sql_str+=")"
    cur.execute("select * from node_freq where node_id in "+sql_str)
    result=cur.fetchall() # [node_id,freq]
    node_freq_dict=dict()
    #print(result)
    for item in result:
        node_freq_dict[item[0]]=item[1]
    is_first=True
    for node_res_item in res_way_node: # node_res_item [way_id,node_id,sequence_id]
        if is_first:
            is_first=False
            continue
        # 查询当前node的出现次数(freq)，如果freq>=2即表示它在其他way中出现过，它是另一条路的交点，应该作为路径端点
        #print(node_res_item)
        curr_node_id=node_res_item[1]
        node_freq=node_freq_dict[curr_node_id]
        #print('node frequency: ',node_freq_res)
        if node_freq>=2: # 当前node应该是路径的终点
            end_node=curr_node_id
            curr_road_nodes.append(end_node)
            road=copy.deepcopy(curr_road_nodes)
            road_list.append(road)
            start_node=end_node
            curr_road_nodes=[start_node]
        else: # 当前node只是路径上的一个转折点
            curr_road_nodes.append(curr_node_id)

    #end=time.perf_counter()
    #print('split node to multiple road time consume: %f seconds'%(end-start))
    # 对road list里面的每条road进行计算
    for r in road_list: # road_list: [[node_id1,node_id2....],[],...]
        #start=time.perf_counter()
        distance_road=calculate_road_length(r)
        #end=time.perf_counter()
        #time_elapse=end-start
        #print('distance calculation consume time: %f seconds'%(time_elapse))
        poiNum=0
        poi_nodes=[] # [[node_id category distance_from_start],....]
        node_from_start_node_list=[]
        # 寻找POI
        sql_str="("
        for n in r:
            sql_str+=str(n)
            sql_str+=','
        sql_str=sql_str[:-1]
        sql_str+=")"
        cur.execute("select * from qualified_poi where node_id in " + sql_str)
        result=cur.fetchall() #[node_id,k,category_id]
        #print(result)
        poi_node_dict=dict()
        if len(result)!=0:
            for item in result:
                poi_node_dict[item[0]]=item[2]
        for n in r: # node id in road 
            node_from_start_node_list.append(n)

            if n in poi_node_dict.keys():
                poiNum+=1
                # search for poi category from database
                #cur.execute('select category_id from qualified_poi where node_id=%d'%(n))
                poi_category=poi_node_dict[n]
                # calculate distance from road start node
                distance_from_start=calculate_road_length(node_from_start_node_list)
                poi_nodes.append([n,poi_category,distance_from_start])

        #print(r[0],r[-1],distance_road,poiNum)
        # 将node的ID进行重新编码，从0开始
        # 首先查询该node是否已经重新编码
        start_node=int(r[0])
        end_node=int(r[-1])

        if start_node in renumbered_node:
            # 已经被重新编码过，在字典中获取之前的编码
            start_node=renumbered_node[start_node]
        else:
            # 没有被重新编码，就赋予新编码
            next_node_id=len(renumbered_node)
            renumbered_node[start_node]=next_node_id
            start_node=next_node_id

        if end_node in renumbered_node:
            # 已经被重新编码过，在字典中获取之前的编码
            end_node=renumbered_node[end_node]
        else:
            # 没有被重新编码，就赋予新编码
            next_node_id=len(renumbered_node)
            renumbered_node[end_node]=next_node_id
            end_node=next_node_id

        map_file.write(str(start_node)+' '+str(end_node)+' '+str(distance_road)
                           +' '+str(poiNum)+'\n')

        if poiNum>0: # that this road contains poi
            for poi in poi_nodes:
                # poi =[node_id,poi_category,distance_from_start,rating]
                rating=random.randint(20,100)
                map_file.write(str(poi[1])+' '+str(poi[2])+' '+str(rating))
            map_file.write('\n')

map_file.close()
for old_node_id in renumbered_node:
    new_node_id=renumbered_node[old_node_id]
    write_latlon(latlon_file,old_node_id,new_node_id)
latlon_file.close()
print("the total node number is %d"%(len(renumbered_node)))

building the 0 th road.
building the 100 th road.
building the 200 th road.
building the 300 th road.
building the 400 th road.
building the 500 th road.
building the 600 th road.
building the 700 th road.
building the 800 th road.
building the 900 th road.
building the 1000 th road.
building the 1100 th road.
building the 1200 th road.
building the 1300 th road.
building the 1400 th road.
building the 1500 th road.
building the 1600 th road.
building the 1700 th road.
building the 1800 th road.
building the 1900 th road.
building the 2000 th road.
building the 2100 th road.
building the 2200 th road.
building the 2300 th road.
building the 2400 th road.
building the 2500 th road.
building the 2600 th road.
building the 2700 th road.
building the 2800 th road.
building the 2900 th road.
building the 3000 th road.
building the 3100 th road.
building the 3200 th road.
building the 3300 th road.
building the 3400 th road.
building the 3500 th road.
building the 3600 th road.
building the 

building the 29700 th road.
building the 29800 th road.
building the 29900 th road.
building the 30000 th road.
building the 30100 th road.
building the 30200 th road.
building the 30300 th road.
building the 30400 th road.
building the 30500 th road.
building the 30600 th road.
building the 30700 th road.
building the 30800 th road.
building the 30900 th road.
building the 31000 th road.
building the 31100 th road.
building the 31200 th road.
building the 31300 th road.
building the 31400 th road.
building the 31500 th road.
building the 31600 th road.
building the 31700 th road.
building the 31800 th road.
building the 31900 th road.
building the 32000 th road.
building the 32100 th road.
building the 32200 th road.
building the 32300 th road.
building the 32400 th road.
building the 32500 th road.
building the 32600 th road.
building the 32700 th road.
building the 32800 th road.
building the 32900 th road.
building the 33000 th road.
building the 33100 th road.
building the 33200 t

building the 59000 th road.
building the 59100 th road.
building the 59200 th road.
building the 59300 th road.
building the 59400 th road.
building the 59500 th road.
building the 59600 th road.
building the 59700 th road.
building the 59800 th road.
building the 59900 th road.
building the 60000 th road.
building the 60100 th road.
building the 60200 th road.
building the 60300 th road.
building the 60400 th road.
building the 60500 th road.
building the 60600 th road.
building the 60700 th road.
building the 60800 th road.
building the 60900 th road.
building the 61000 th road.
building the 61100 th road.
building the 61200 th road.
building the 61300 th road.
building the 61400 th road.
building the 61500 th road.
building the 61600 th road.
building the 61700 th road.
building the 61800 th road.
building the 61900 th road.
building the 62000 th road.
building the 62100 th road.
building the 62200 th road.
building the 62300 th road.
building the 62400 th road.
building the 62500 t

building the 88300 th road.
building the 88400 th road.
building the 88500 th road.
building the 88600 th road.
building the 88700 th road.
building the 88800 th road.
building the 88900 th road.
building the 89000 th road.
building the 89100 th road.
building the 89200 th road.
building the 89300 th road.
building the 89400 th road.
building the 89500 th road.
building the 89600 th road.
building the 89700 th road.
building the 89800 th road.
building the 89900 th road.
building the 90000 th road.
building the 90100 th road.
building the 90200 th road.
building the 90300 th road.
building the 90400 th road.
building the 90500 th road.
building the 90600 th road.
building the 90700 th road.
building the 90800 th road.
building the 90900 th road.
building the 91000 th road.
building the 91100 th road.
building the 91200 th road.
building the 91300 th road.
building the 91400 th road.
building the 91500 th road.
building the 91600 th road.
building the 91700 th road.
building the 91800 t

building the 117000 th road.
building the 117100 th road.
building the 117200 th road.
building the 117300 th road.
building the 117400 th road.
building the 117500 th road.
building the 117600 th road.
building the 117700 th road.
building the 117800 th road.
building the 117900 th road.
building the 118000 th road.
building the 118100 th road.
building the 118200 th road.
building the 118300 th road.
building the 118400 th road.
building the 118500 th road.
building the 118600 th road.
building the 118700 th road.
building the 118800 th road.
building the 118900 th road.
building the 119000 th road.
building the 119100 th road.
building the 119200 th road.
building the 119300 th road.
building the 119400 th road.
building the 119500 th road.
building the 119600 th road.
building the 119700 th road.
building the 119800 th road.
building the 119900 th road.
building the 120000 th road.
building the 120100 th road.
building the 120200 th road.
building the 120300 th road.
building the 1

building the 145300 th road.
building the 145400 th road.
building the 145500 th road.
building the 145600 th road.
building the 145700 th road.
building the 145800 th road.
building the 145900 th road.
building the 146000 th road.
building the 146100 th road.
building the 146200 th road.
building the 146300 th road.
building the 146400 th road.
building the 146500 th road.
building the 146600 th road.
building the 146700 th road.
building the 146800 th road.
building the 146900 th road.
building the 147000 th road.
building the 147100 th road.
building the 147200 th road.
building the 147300 th road.
building the 147400 th road.
building the 147500 th road.
building the 147600 th road.
building the 147700 th road.
building the 147800 th road.
building the 147900 th road.
building the 148000 th road.
building the 148100 th road.
building the 148200 th road.
building the 148300 th road.
building the 148400 th road.
building the 148500 th road.
building the 148600 th road.
building the 1

building the 173600 th road.
building the 173700 th road.
building the 173800 th road.
building the 173900 th road.
building the 174000 th road.
building the 174100 th road.
building the 174200 th road.
building the 174300 th road.
building the 174400 th road.
building the 174500 th road.
building the 174600 th road.
building the 174700 th road.
building the 174800 th road.
building the 174900 th road.
building the 175000 th road.
building the 175100 th road.
building the 175200 th road.
building the 175300 th road.
building the 175400 th road.
building the 175500 th road.
building the 175600 th road.
building the 175700 th road.
building the 175800 th road.
building the 175900 th road.
building the 176000 th road.
building the 176100 th road.
building the 176200 th road.
building the 176300 th road.
building the 176400 th road.
building the 176500 th road.
building the 176600 th road.
building the 176700 th road.
building the 176800 th road.
building the 176900 th road.
building the 1

building the 201900 th road.
building the 202000 th road.
building the 202100 th road.
building the 202200 th road.
building the 202300 th road.
building the 202400 th road.
building the 202500 th road.
building the 202600 th road.
building the 202700 th road.
building the 202800 th road.
building the 202900 th road.
building the 203000 th road.
building the 203100 th road.
building the 203200 th road.
building the 203300 th road.
building the 203400 th road.
building the 203500 th road.
building the 203600 th road.
building the 203700 th road.
building the 203800 th road.
building the 203900 th road.
building the 204000 th road.
building the 204100 th road.
building the 204200 th road.
building the 204300 th road.
building the 204400 th road.
building the 204500 th road.
building the 204600 th road.
building the 204700 th road.
building the 204800 th road.
building the 204900 th road.
building the 205000 th road.
building the 205100 th road.
building the 205200 th road.
building the 2

building the 230200 th road.
building the 230300 th road.
building the 230400 th road.
building the 230500 th road.
building the 230600 th road.
building the 230700 th road.
building the 230800 th road.
building the 230900 th road.
building the 231000 th road.
building the 231100 th road.
building the 231200 th road.
building the 231300 th road.
building the 231400 th road.
building the 231500 th road.
building the 231600 th road.
building the 231700 th road.
building the 231800 th road.
building the 231900 th road.
building the 232000 th road.
building the 232100 th road.
building the 232200 th road.
building the 232300 th road.
building the 232400 th road.
building the 232500 th road.
building the 232600 th road.
building the 232700 th road.
building the 232800 th road.
building the 232900 th road.
building the 233000 th road.
building the 233100 th road.
building the 233200 th road.
building the 233300 th road.
building the 233400 th road.
building the 233500 th road.
building the 2

# 优化方法

* 尽可能在数据库内部进行运算，将计算结果保存为新表，不要逐个取出数据再运算
* 执行多条sql后再commit，不要每执行一条查询就commit一次
* 使用in而不是多个条件or
* 多个筛选条件放在一起使用where子句带in或者exist解决
* 多条数据需要查询的时候，先全部查询出来，保存在一个dict里面。而不是每条都发起一个query
* 几万到十几万条的数据使用dict保存而不是通过数据库查询

# playground

In [35]:
conn.rollback()

In [38]:
%%time
map_file=open('./DATA_MI_0.txt','w')
latlon_file=open('./LATLON_MI_0.txt','w')
#res_qualified_way_id=[[192046993,]]
count=0
renumbered_node=dict()
#for row in res_qualified_way_id:
row=[13626616]
if count%100==0:
    print('building the %d th road.'%(count))
count+=1

start=time.perf_counter()
qualified_way_id=row[0]
cur.execute("select * from qualified_way_nodes where way_id=%d order by sequence_id"%(qualified_way_id))
res_way_node=cur.fetchall() # [[way_id,node_id,sequenced_id],[],[]]
end=time.perf_counter()
print("time for select way nodes:",end-start)

isFirst=True
start_node=0
start_node_renumber=0
start_n_lonlat=[0,0,0]
end_node=0
end_n_lon_lat=[0,0,0]
end_node_renumber=0
#print(res_way_node)
for item in res_way_node:
    if isFirst:
        start_node=item[1]
        cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id ="+str(start_node))
        result=cur.fetchall()
        start_n_lonlat=result[0]
        if start_node in renumbered_node:
            # 已经被重新编码过，在字典中获取之前的编码
            start_node_renumber=renumbered_node[start_node]
        else:
            # 没有被重新编码，就赋予新编码
            next_node_id=len(renumbered_node)
            renumbered_node[start_node]=next_node_id
            start_node_renumber=next_node_id
        latlon_file.write(str(start_node_renumber)+" "+str(start_n_lonlat[1])+" "+str(start_n_lonlat[2])+"\n")
        isFirst=False
        continue
    end_node=item[1]
    
    start=time.perf_counter()
    cur.execute("select id, st_x(geom) as lon, st_y(geom) as lat from qualified_nodes where id ="+str(end_node))
    result=cur.fetchall()
    end=time.perf_counter()
    print("time for select lonlat:",end-start)
    end_n_lonlat=result[0]
    node_list=[start_node,end_node]
    print(start_n_lonlat,"   ",end_n_lonlat)
    
    start=time.perf_counter()
    distance=haversine(start_n_lonlat[1],start_n_lonlat[2],end_n_lonlat[1],end_n_lonlat[2])
    end=time.perf_counter()
    print("time for computing distance:",end-start)
    
    start=time.perf_counter()
    if end_node in renumbered_node:
        # 已经被重新编码过，在字典中获取之前的编码
        end_node_renumber=renumbered_node[end_node]
    else:
        # 没有被重新编码，就赋予新编码
        next_node_id=len(renumbered_node)
        renumbered_node[end_node]=next_node_id
        end_node_renumber=next_node_id
    end=time.perf_counter()
    print("time for renumber:",end-start)
        
    map_file.write(str(start_node_renumber)+' '+str(end_node_renumber)+' '+str(distance)
                       +' '+'0'+'\n')
    latlon_file.write(str(end_node_renumber)+" "+str(end_n_lonlat[1])+" "+str(end_n_lonlat[2])+"\n")    
    start_node=end_node
    start_n_lonlat=end_n_lonlat
    start_node_renumber=end_node_renumber
      

map_file.close()
latlon_file.close()
print("the total node number is %d"%(len(renumbered_node)))
        

building the 0 th road.
time for select way nodes: 0.13330610000002707
time for select lonlat: 0.1192364999997153
(125037565, -88.5984526, 30.7349923)     (125383414, -88.5984553, 30.7345243)
time for computing distance: 1.3700000181415817e-05
time for renumber: 1.8000000636675395e-06
time for select lonlat: 0.11202510000020993
(125383414, -88.5984553, 30.7345243)     (125383416, -88.598463, 30.733322)
time for computing distance: 1.0299999757990008e-05
time for renumber: 1.8000000636675395e-06
time for select lonlat: 0.12483209999982137
(125383416, -88.598463, 30.733322)     (125383419, -88.598451, 30.732925)
time for computing distance: 1.3600000329461182e-05
time for renumber: 1.99999976757681e-06
time for select lonlat: 0.11832099999992352
(125383419, -88.598451, 30.732925)     (125383421, -88.59841, 30.732471)
time for computing distance: 1.0299999757990008e-05
time for renumber: 1.8999999156221747e-06
time for select lonlat: 0.13027969999984634
(125383421, -88.59841, 30.732471)  

time for select lonlat: 0.11436289999983273
(125383512, -88.581907, 30.709186)     (125383515, -88.579785, 30.706492)
time for computing distance: 1.0100000054080738e-05
time for renumber: 1.8999999156221747e-06
time for select lonlat: 0.11399990000018079
(125383515, -88.579785, 30.706492)     (125383517, -88.579552, 30.706188)
time for computing distance: 9.299999874201603e-06
time for renumber: 3.3999999686784577e-06
time for select lonlat: 0.1112822000000051
(125383517, -88.579552, 30.706188)     (125383522, -88.579452, 30.706043)
time for computing distance: 1.0300000212737359e-05
time for renumber: 1.5999999050109182e-06
time for select lonlat: 0.1109747000000425
(125383522, -88.579452, 30.706043)     (125383524, -88.579242, 30.70572)
time for computing distance: 9.800000043469481e-06
time for renumber: 1.5999999050109182e-06
time for select lonlat: 0.11546960000032414
(125383524, -88.579242, 30.70572)     (125383526, -88.578957, 30.705217)
time for computing distance: 1.019999990

time for select lonlat: 0.11455310000019381
(125383637, -88.576377, 30.682724)     (125383639, -88.576335, 30.682347)
time for computing distance: 1.0199999906035373e-05
time for renumber: 1.7999996089201886e-06
time for select lonlat: 0.11867389999997613
(125383639, -88.576335, 30.682347)     (125383646, -88.576267, 30.681935)
time for computing distance: 1.6400000276917126e-05
time for renumber: 3.000000106112566e-06
time for select lonlat: 0.12220269999988886
(125383646, -88.576267, 30.681935)     (125383652, -88.576191, 30.681551)
time for computing distance: 1.0100000054080738e-05
time for renumber: 1.8999999156221747e-06
time for select lonlat: 0.110877900000105
(125383652, -88.576191, 30.681551)     (125383656, -88.576125, 30.681262)
time for computing distance: 1.0100000054080738e-05
time for renumber: 1.6999997569655534e-06
time for select lonlat: 0.1131164000003082
(125383656, -88.576125, 30.681262)     (125383661, -88.575967, 30.680706)
time for computing distance: 1.0300000

time for select lonlat: 0.11316480000004958
(125383754, -88.565429, 30.661792)     (125383755, -88.565052, 30.661135)
time for computing distance: 1.1000000085914508e-05
time for renumber: 2.4000000848900527e-06
time for select lonlat: 0.11771600000020044
(125383755, -88.565052, 30.661135)     (125383760, -88.564681, 30.660469)
time for computing distance: 9.800000043469481e-06
time for renumber: 1.8000000636675395e-06
time for select lonlat: 0.11831039999970017
(125383760, -88.564681, 30.660469)     (125383770, -88.564626, 30.660362)
time for computing distance: 1.900000006571645e-05
time for renumber: 3.000000106112566e-06
time for select lonlat: 0.11875420000023951
(125383770, -88.564626, 30.660362)     (7800190635, -88.5645772, 30.6602637)
time for computing distance: 9.700000191514846e-06
time for renumber: 1.8000000636675395e-06
time for select lonlat: 0.11185669999986203
(7800190635, -88.5645772, 30.6602637)     (7800190631, -88.5645257, 30.6601601)
time for computing distance: 

time for select lonlat: 0.11950630000001183
(125383882, -88.567267, 30.647081)     (125383886, -88.567305, 30.646981)
time for computing distance: 1.720000000204891e-05
time for renumber: 3.3999999686784577e-06
time for select lonlat: 0.12237659999982498
(125383886, -88.567305, 30.646981)     (125383891, -88.567394, 30.646724)
time for computing distance: 8.999999863590347e-06
time for renumber: 2.100000074278796e-06
time for select lonlat: 0.12343980000014199
(125383891, -88.567394, 30.646724)     (125383894, -88.567579, 30.646119)
time for computing distance: 9.59999988481286e-06
time for renumber: 1.9000003703695256e-06
time for select lonlat: 0.11318460000029518
(125383894, -88.567579, 30.646119)     (125383896, -88.567673, 30.645769)
time for computing distance: 8.900000011635711e-06
time for renumber: 1.8000000636675395e-06
time for select lonlat: 0.11980719999974099
(125383896, -88.567673, 30.645769)     (125383900, -88.567785, 30.645278)
time for computing distance: 1.879999990

time for select lonlat: 0.11957590000019991
(7307837432, -88.5626187, 30.6203843)     (6948067481, -88.562463, 30.6197898)
time for computing distance: 1.849999989644857e-05
time for renumber: 3.599999672587728e-06
time for select lonlat: 0.11861410000028627
(6948067481, -88.562463, 30.6197898)     (125384043, -88.562411, 30.619591)
time for computing distance: 9.100000170292333e-06
time for renumber: 2.1999999262334313e-06
time for select lonlat: 0.11549460000014733
(125384043, -88.562411, 30.619591)     (125384046, -88.562283, 30.619127)
time for computing distance: 9.100000170292333e-06
time for renumber: 2.100000074278796e-06
time for select lonlat: 0.11428290000003472
(125384046, -88.562283, 30.619127)     (125355906, -88.562207, 30.618796)
time for computing distance: 9.40000018090359e-06
time for renumber: 1.8000000636675395e-06
time for select lonlat: 0.11526400000002468
(125355906, -88.562207, 30.618796)     (125384048, -88.562134, 30.618414)
time for computing distance: 1.010

time for select lonlat: 0.11752589999969132
(7115822480, -88.5649993, 30.6057841)     (125350736, -88.564721, 30.604252)
time for computing distance: 1.049999991664663e-05
time for renumber: 1.8000000636675395e-06
time for select lonlat: 0.1183338000000731
(125350736, -88.564721, 30.604252)     (125384198, -88.564466, 30.602824)
time for computing distance: 9.699999736767495e-06
time for renumber: 2.2999997781880666e-06
time for select lonlat: 0.12151519999997618
(125384198, -88.564466, 30.602824)     (125384200, -88.564391, 30.602444)
time for computing distance: 9.299999874201603e-06
time for renumber: 1.9000003703695256e-06
time for select lonlat: 0.11832800000001953
(125384200, -88.564391, 30.602444)     (7872065602, -88.564304, 30.6019544)
time for computing distance: 1.44999999065476e-05
time for renumber: 3.1999998100218363e-06
time for select lonlat: 0.11357109999971726
(7872065602, -88.564304, 30.6019544)     (7872065603, -88.5642457, 30.6016267)
time for computing distance: 1

time for select lonlat: 0.12344490000032238
(125383133, -88.561985, 30.579855)     (7983323205, -88.5625122, 30.5777735)
time for computing distance: 9.40000018090359e-06
time for renumber: 2.100000074278796e-06
time for select lonlat: 0.11757390000002488
(7983323205, -88.5625122, 30.5777735)     (125383134, -88.563186, 30.575113)
time for computing distance: 1.699999984339229e-05
time for renumber: 3.999999989900971e-06
time for select lonlat: 0.1144030999998904
(125383134, -88.563186, 30.575113)     (125383141, -88.564607, 30.569489)
time for computing distance: 9.299999874201603e-06
time for renumber: 1.99999976757681e-06
time for select lonlat: 0.5028049999996256
(125383141, -88.564607, 30.569489)     (125383144, -88.565442, 30.566186)
time for computing distance: 1.0199999906035373e-05
time for renumber: 2.099999619531445e-06
time for select lonlat: 0.5027092000000266
(125383144, -88.565442, 30.566186)     (125383145, -88.565538, 30.56577)
time for computing distance: 1.1000000085

time for select lonlat: 0.12020910000001095
(7635214841, -88.5463213, 30.5289737)     (125335384, -88.545922, 30.528292)
time for computing distance: 1.140000040322775e-05
time for renumber: 2.200000380980782e-06
time for select lonlat: 0.11399789999995846
(125335384, -88.545922, 30.528292)     (125384295, -88.544691, 30.526191)
time for computing distance: 9.200000022246968e-06
time for renumber: 2.100000074278796e-06
time for select lonlat: 0.11371730000018943
(125384295, -88.544691, 30.526191)     (125384298, -88.544215, 30.525411)
time for computing distance: 1.0799999927257886e-05
time for renumber: 2.1999999262334313e-06
time for select lonlat: 0.12374580000005153
(125384298, -88.544215, 30.525411)     (125353676, -88.543891, 30.524902)
time for computing distance: 2.099999983329326e-05
time for renumber: 4.399999852466863e-06
time for select lonlat: 0.1213700999996945
(125353676, -88.543891, 30.524902)     (125384301, -88.543483, 30.524268)
time for computing distance: 2.5299999

In [28]:
latlon_file.close()